In [ ]:
#bing 에서 연예인 이미지 가져오기

!git clone https://github.com/ndb796/bing_image_downloader

In [ ]:
import os
import shutil
from bing_image_downloader.bing_image_downloader import downloader

directory_list = [
    './custom_dataset/train/',
    './custom_dataset/test/',
]

#데이터셋 저장 디렉토리 생성
for directory in directory_list:
    if not os.path.isdir(directory):
        os.makedirs(directory)


In [ ]:

#수집한 데이터를 학습데이터와 평가데이터로 분류하는 함수

def dataset_split(query , train_cnt):

    for directory in directory_list:
        if not os.path.isdir(directory + '/' + query):
            os.makedirs(directory + '/' + query)


    cnt = 0

    for file_name in os.listdir(query):
        if cnt < train_cnt:
            print(f'[Train Dataset] {file_name} ')
            shutil.move(query + '/' + file_name , './custom_dataset/train/'+query +'/' +file_name)
        else:
            print(f'[Test Dataset] {file_name} ')
            shutil.move(query + '/' + file_name , './custom_dataset/test/'+query +'/' +file_name)
        cnt += 1
    
    shutil.rmtree(query)

In [ ]:
query = '카리나'
downloader.download(query , limit=40, output_dir = './', adult_filter_off=True,force_replace=False,timeout=60)
dataset_split(query,30)

In [ ]:
query = '마동석'
downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
dataset_split(query, 30)

In [ ]:
query = '이국주'
downloader.download(query, limit=40,  output_dir='./', adult_filter_off=True, force_replace=False, timeout=60)
dataset_split(query, 30)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets,models,transforms

import numpy as np
import time

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#데이터셋을 불러올때 상요할 변형정의
transforms_train = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

transforms_test = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [ ]:
train_datasets = datasets.ImageFolder('./custom_dataset/train',transforms_train)
test_datasets = datasets.ImageFolder('./custom_dataset/test',transforms_test)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(train_datasets,batch_size=4,shuffle=True,num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_datasets,batch_size=4,shuffle=True,num_workers=2)

print('학습데이터셋 크기 : ', len(train_datasets))
print('테스트데이터셋 크기:',len(test_datasets))

class_names = train_datasets.classes
print('클래스:',class_names)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def imshow(input, title):
    #torch.Tensor를 numpy객체로 변환
    print(input.numpy().shape)

    input = input.numpy().transpose((1,2,0))

    mean = np.array([0.485,0.456,0.406])
    std = np.array([0.299,0.224,0.225])

    input = std * input + mean
    input= np.clip(input,0,1)

    print('input shpae : ' ,input.shape )
    print('title : ',title)
    
    plt.imshow(input)
    plt.title(title)
    plt.show()
    

    

In [ ]:
iterator = iter(train_dataloader)

inputs, classes = next(iterator)
print(classes)
print(inputs.shape)
out = torchvision.utils.make_grid(inputs)
imshow(out,title=[ class_names[x] for x in classes ])

In [ ]:
model = models.resnet34(pretrained=True)
model

In [ ]:

for param in model.parameters():
    param.requires_grad = False

num_features = model.fc.in_features
model.fc = nn.Linear(num_features,3)
model = model.to(device)

In [ ]:
for name,module in model.named_parameters():
    print(name + "  :  " + str( module.requires_grad ))


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters() , lr=0.001 , momentum=0.9)

In [ ]:
num_epochs = 50
model.train()
start_time = time.time()


for epoch in range(num_epochs):
    running_loss = 0.
    running_corrects = 0

    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        _,preds = torch.max(outputs,1)
        loss = criterion(outputs,labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(train_datasets)
    epoch_acc = running_corrects / len(train_datasets) * 100.

    print('#{} Loss:{:.4f} Acc:{:.4f}% Time:{:.4f}s'.format(epoch,epoch_loss,epoch_acc,time.time()-start_time))
